This notebook aims to update the commercial and residential listings in accordance with their respective zones from the city zone map.

In [8]:
!pip install shapely

DEPRECATION: Loading egg at /Users/habib/miniconda3/envs/pytorch_m1_3.11/lib/python3.11/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [23]:
# print the zonal map data
import pandas as pd
import numpy as np
from shapely.geometry import MultiPolygon, Point
from shapely.wkt import loads as load_wkt

# read the data
zonal_data = pd.read_csv('../data/Zoning_Bylaw_Geographical_Data_20240528.csv')

# print the data
print(zonal_data.head(5))

       id  zoning               description    date_ext  \
0  169754      PS        Parks and Services  2024/05/27   
1  175633      CN  Neighbourhood Commercial  2024/05/27   
2  170985      CG        General Commercial  2024/05/27   
3  176327  RM h16  Medium Scale Residential  2024/05/27   
4  176677  RM h16  Medium Scale Residential  2024/05/27   

                                                 url  \
0  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
1  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
2  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
3  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   
4  http://webdocs.edmonton.ca/InfraPlan/zoningbyl...   

                               geometry_multipolygon  
0  MULTIPOLYGON (((-113.46405030242478 53.4139561...  
1  MULTIPOLYGON (((-113.48544480430161 53.5005298...  
2  MULTIPOLYGON (((-113.41637166582207 53.5855382...  
3  MULTIPOLYGON (((-113.49564121741932 53.5688278...  
4  MULTIPOLYGON (((-113.377186670

In [24]:
# Function to check if a point lies within a multipolygon
def is_point_in_multipolygon(lat, lon):
    point = Point(lon, lat)  # Note: Shapely uses (lon, lat) order
    for _, row in zonal_data.iterrows():
        multipolygon_wkt = row['geometry_multipolygon']
        multipolygon = load_wkt(multipolygon_wkt)
        if multipolygon.contains(point):
            # return as row as json
            return row.to_dict()
    return {}

# Test the function with an example point
lat = 53.5005
lon = -113.4855

print({'lat': lat, 'lon': lon, 'zoning': is_point_in_multipolygon(lat, lon)})

{'lat': 53.5005, 'lon': -113.4855, 'zoning': {'id': 175633, 'zoning': 'CN', 'description': 'Neighbourhood Commercial', 'date_ext': '2024/05/27', 'url': 'http://webdocs.edmonton.ca/InfraPlan/zoningbylaw/bylaw_12800.htm', 'geometry_multipolygon': 'MULTIPOLYGON (((-113.48544480430161 53.500529804717324, -113.48575021281256 53.50071076505371, -113.48624647281675 53.50100695034535, -113.4862309463645 53.49987387916919, -113.48469211617721 53.49987363830927, -113.48469421839806 53.50005511947766, -113.4846925282687 53.50040641492085, -113.4852351676 53.50040558977945, -113.48544480430161 53.500529804717324)))'}}


In [25]:
# read the {} data and find its zone and update the data
# residential datas are in json format and in ../data/{}}/ folder they are slplited into multiple .js files

def update_zoning(listing_type):
    import json
    import os

    for file in os.listdir(f'../data/{listing_type}/'):
        with open(f'../data/{listing_type}/' + file) as f:
            data = json.load(f)
            listings = data['Results']
            
            for listing in listings:
                lat = listing['Property']['Address']['Latitude']
                lon = listing['Property']['Address']['Longitude']
                
                # Find the zoning for the listing
                zone = is_point_in_multipolygon(lat, lon)
                listing['zone'] = zone
            
            # Save the updated data with _mapped suffix in the filename
            with open(f'../data/{listing_type}/' + file.replace('.js', '_zone_mapped.js'), 'w') as newfile:
                json.dump(data, newfile)
                newfile.close()
            
            f.close()
            
# Update the zoning for residential and commercial listings
update_zoning('residential')
update_zoning('commercial')